In [ ]:
import logging
import pandas as pd
import numpy as np
import gensim
import nltk
import re
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings ('ignore')

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

def clean_text(text):
    if type(text)!=str:
        return ""
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [ ]:
#from itertools import islice
wv = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
wv.init_sims(replace=True)
wv.fill_norms()

In [ ]:
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import gmean

def word_averaging(wv, words):
    all_words, mean = set(), []
    cou=0
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.key_to_index:
            mean.append(wv.vectors[wv.key_to_index[word]])
            all_words.add(wv.key_to_index[word])

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)
    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    #mean = gensim.matutils.unitvec(gmean(np.array(mean))).astype(np.float32)
    #mean = gensim.matutils.unitvec(np.percentile(np.array(mean),25,axis=0)).astype(np.float32)
    #mean = gensim.matutils.unitvec(skew(np.array(mean))).astype(np.float32)
   #mean = gensim.matutils.unitvec(np.percentile(np.array(mean),50,axis=0)).astype(np.float32)
    return mean

In [ ]:
def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            tokens.append(word)
    return tokens

In [ ]:
fname='GitterCom'+'.csv'
df = pd.read_csv(fname,encoding='utf8')
df.drop([581, 2102, 4858, 6417], axis=0, inplace=True)

non_string = []
for i, row in df.iterrows():
    res = ((type(row['message']) != str) or (type(row['Purpose']) != str) or (type(row['Category']) != str) or (type(row['Subcategory']) != str))
    if res:
        non_string.append(i)

df.drop(non_string, axis=0, inplace=True)
len(df)

9959

In [ ]:
df.drop(['Channel','messageId','time','user'],axis=1,inplace=True)
df['Category']=df['Category'].str.lower()
df['Purpose']=df['Purpose'].str.lower()
df['Subcategory']=df['Subcategory'].str.lower()
df['message'] = df['message'].apply(clean_text)
df.head(10)

,message,Purpose,Category,Subcategory
0,hi team recently upgraded cucumberjvm version ...,team wide,dev-ops,development operation notifications
1,exception thread main javalangnosuchmethoderro...,team wide,dev-ops,development operation notifications
2,amit007 looks like inconsistent cucumber jar v...,team wide,communication,communication with teammates
3,github trying replace irc p,team wide,communication,communication with teammates
4,aslakhellesoy thanks seems like using older ve...,team wide,communication,communication with teammates
5,hi one question friends starting project studi...,team wide,dev-ops,team q&a
6,danon9111 integrate cucumber another testing f...,team wide,dev-ops,team q&a
7,aslakhellesoy reading cucumber book found code...,team wide,dev-ops,team q&a
8,sidkiyassine call methods directly,team wide,dev-ops,team q&a
9,intergrated cucumber+appium https githubcom pr...,team wide,dev-ops,development operation notifications


In [ ]:
comtdata=df
test_tokenized = comtdata.apply(lambda r: w2v_tokenize_text(r['message']), axis=1).values
X_comtdata_average1 = word_averaging_list(wv,test_tokenized)
fname='6'+'.csv'   #w2v
np.savetxt(fname,X_comtdata_average1, delimiter=',', fmt='%f')